# EE123 Final Project Sp20 - Video Communication
### Bryan Yang and Lixing He

In [ ]:
# Import functions and libraries (Part I)
import numpy as np
import scipy.fftpack
from scipy.fftpack import dct, idct
import matplotlib.pyplot as plt
from bitarray import bitarray
from huffman import *
from os import stat
from PIL import Image


# Import functions and libraries (Part II)
import queue as Queue
import threading,time
import sys

from numpy import pi
from numpy import sin
from numpy import zeros
from numpy import r_
from numpy import ones
from scipy import signal
from scipy import integrate

import time
import multiprocessing

from numpy import mean
from numpy import power
from numpy.fft import fft
from numpy.fft import fftshift
from numpy.fft import ifft
from numpy.fft import ifftshift
from  scipy.io.wavfile import read as wavread
#import ax25
from math import gcd
import sounddevice as sd
import RPi.GPIO as GPIO
from functools import reduce
from numpy import ones,zeros, pi, cos, exp, sign


# Import functions and libraries (Part III)
import base64
import threading
import newax25 as ax25


%matplotlib inline

###### Commands to run

Compile PLL

In [ ]:
!cc -fPIC -shared -o libpll.so pll.c

Set up loopback

In [ ]:
!sudo modprobe snd_aloop
!aplay -L | grep -i plughw:card=loopback
!aplay -l | grep -i loopback

In [ ]:
sd.query_devices()

In [ ]:
builtin_idx = 0
usb_idx = 2
loop_idx = 4  # Use loopback dev 1! direwolf is connected to dev 0.
loop_idx_rcv = 3 # Use loopback dev 0. 
sd.default.samplerate=48000
sd.default.channels = 1

Set up direwolf

In [ ]:
# Enter your address here
myaddress = "Cory Hall Berkeley CA USA"

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="EE123")
location = geolocator.geocode(myaddress)

In [ ]:
# hopefully it did not fail
print(location.address)
print((location.latitude, location.longitude))

In [ ]:
import math
lat = "lat=%d^%2.2f%s" % (int(abs(location.latitude)),
                          60*(abs(location.latitude) - abs(math.floor(location.latitude))),
                          ("N") if location.latitude >0 else "S" )  
long = "long=%d^%2.2f%s" % (int(abs(location.longitude)),
                            60*(abs(location.longitude) - math.floor(abs(location.longitude))),
                          ("W") if location.longitude <0 else "E" )  
                            
print(lat, long)

In [ ]:
# enter your callsign and comment for the beacon

callsign = "KN6IDV-1"
comment = "EE123 is cool"


callsignr = callsign.split('-')[0].upper()
code = 0x73e2
for i, char in enumerate(callsignr):
    code ^= ord(char) << (8 if not i % 2 else 0)

passcode = code&0x7fff
print("Passcode:", passcode)

In [ ]:
import os

cmd = "cat direwolf-loopback-DRAFT.conf  | sed  's/EE123_CALL/"+callsign+"/g'  | "
cmd = cmd +  "sed  's/EE123_PASSCODE/%d/g' | " % (passcode)
cmd = cmd +  "sed  's/EE123_COMMENT/comment=\"%s\"/g' | " % (comment)
cmd = cmd +  "sed  's/EE123_LATCOORD/%s/g' | " % (lat)
cmd = cmd +  "sed  's/EE123_LONGCOORD/%s/g' > direwolf-loopback.conf" % (long)
print(cmd)
os.system(cmd)  # NOTE: this is poor security practice, but conveniently concise here